# Text Summarization: 01. Raw data.

* Model: Sequence-to-sequence model
    * decode-only model needs examples: one shot or few shot.
    * Potential baseline model: the first three sentences.
    * T5 or Flan T5?
        * it has prefix: `summarize:`, `document:`, versatile, https://huggingface.co/docs/transformers/model_doc/t5
        * https://huggingface.co/docs/transformers/model_doc/flan-t5
* Data source
    * Longform Article summarization: https://huggingface.co/datasets/vgoldberg/longform_article_summarization?row=0 (too long)
    * CNN daily mail: https://huggingface.co/datasets/cnn_dailymail average token 781, summary: 56 tokens, download last months: 94k
    * Bill Summary: https://huggingface.co/datasets/dreamproit/bill_summary_us , text: 4,687 chars, summary: 640 chars, download: 3
    * Medical meadow: https://huggingface.co/datasets/medalpaca/medical_meadow_cord19, from abstract to title, download: 127 (sometimes: non-english)
    * Chemical summary: https://huggingface.co/datasets/griffin/ChemSum, from abstract to title, download: 122
    * Selected Bill Summary data
* Data Analysis: the number of tokens
* Metrics: ROUGE score
* instruction fine tuning with QLORA
* Data Loading: dataset or iterable dataset: https://huggingface.co/docs/datasets/about_mapstyle_vs_iterable

* CUDA install
    1. Use additional driver GUI
    2. type nvidia-smi to check cuda version
    3. install pytorch according to cuda version

## Import libraries

In [2]:
import os
from datasets import load_dataset, list_datasets, DatasetDict

In [3]:
# set variables
SEED = 1234
curr_path = os.getcwd()
path_raw = os.path.join(curr_path, '../data/raw', 'bill_summary_us')
# dataset_name = 'vgoldberg/longform_article_summarization'
dataset_name = 'dreamproit/bill_summary_us'

# load dataset
* Current dataset has only train
* Split train val test
* Need DatasetDict

In [4]:
# list datasets
dataset_list = list_datasets()
if dataset_name in dataset_list:
    dataset = load_dataset(dataset_name)
print(dataset)

/tmp/ipykernel_41148/3008630791.py:2: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  dataset_list = list_datasets()


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'congress', 'bill_type', 'bill_number', 'bill_version', 'sections', 'sections_length', 'text', 'text_length', 'summary', 'summary_length', 'title'],
        num_rows: 125246
    })
})


## Split train, dev, test

In [5]:
# train, dev, test: 80%, 10%, 10%
dataset_train_devtest = dataset['train'].train_test_split(train_size=0.8, shuffle=True, seed=SEED)
dataset_dev_test = dataset_train_devtest['test'].train_test_split(train_size=0.5, shuffle=True, seed=SEED)
dataset_splits = DatasetDict(
    {
        'train': dataset_train_devtest['train'], 
        'dev': dataset_dev_test['train'], 
        'test': dataset_dev_test['test']
    }
)
print(dataset_splits)

DatasetDict({
    train: Dataset({
        features: ['id', 'congress', 'bill_type', 'bill_number', 'bill_version', 'sections', 'sections_length', 'text', 'text_length', 'summary', 'summary_length', 'title'],
        num_rows: 100196
    })
    dev: Dataset({
        features: ['id', 'congress', 'bill_type', 'bill_number', 'bill_version', 'sections', 'sections_length', 'text', 'text_length', 'summary', 'summary_length', 'title'],
        num_rows: 12525
    })
    test: Dataset({
        features: ['id', 'congress', 'bill_type', 'bill_number', 'bill_version', 'sections', 'sections_length', 'text', 'text_length', 'summary', 'summary_length', 'title'],
        num_rows: 12525
    })
})


## Save to disk for later use

In [6]:
# Save
dataset_splits.save_to_disk(path_raw)
print(f'path to the raw data: {path_raw}')
!ls {path_raw}

Saving the dataset (0/6 shards):   0%|          | 0/100196 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12525 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12525 [00:00<?, ? examples/s]

path to the raw data: /media/stephen/HDD2/Document/Research/document_summarization/summarization/notebooks/../data/raw/bill_summary_us
dataset_dict.json  dev	test  train
